Post Processing of Model output

<font size="5">Run the ANUGA Model (BYOAM) </font>

<font size="3">In this notebook, we will:

- initiate the model with elevation, friction, stage
- set the boundary conditions
- run the model!



</font>

<font size="3">Before you can run this notebook, you should have run the 4_Build_ANUGA_Mesh</font>


<font size=5> Step #1: Mount Google Drive and Grant Access <font> <br>




<font size=3> When you run the next cell, a pop-up window will appear asking you to grant access to your Google Drive. You must approve or the notebook will not work. <font> <br>


In [ ]:
import sys
import os
path = os.getcwd() + '/'
where_to_mount = '/content/drive/'

from google.colab import drive
drive.mount(where_to_mount, force_remount=True)

mounted_drive = where_to_mount + 'MyDrive/'

<font size=5> Step #2: Install packages. <font> <br>
<font size=3> This cell should install all Python packages you need for each tutorial.<font>

In [ ]:
import os
os.chdir(mounted_drive)
if os.path.isdir(mounted_drive + 'komo_estuary_tutorial'):
  print('## Updating the local git repository \n')
  os.chdir(mounted_drive + '/komo_estuary_tutorial')
  ! git pull 
else:
  print('## Pulling the git repository with files for the tutorial\n')
  ! git clone https://github.com/achri19/komo_estuary_tutorial.git

print('\n## Installing the Python packages needed for these tutorials\n')
!/bin/bash $mounted_drive/komo_estuary_tutorial/install_packages.sh


## Updating the local git repository 

Updating 8b44f61..9654887
error: Your local changes to the following files would be overwritten by merge:
	4_Build_ANUGA_Mesh.ipynb
Please commit your changes or stash them before you merge.
Aborting

## Installing the Python packages needed for these tutorials

/content/drive/MyDrive//komo_estuary_tutorial/install_packages.sh: line 4: cd: /content/drive/MyDrive/installations: No such file or directory
(1) Install pip packages
nose mpi4py triangle Pmw pymetis cmocean geopandas fiona pygeos rasterio rasterstats
(2) Install gdal
(3) Install netcdf4
(4) Download anuga_core github repository
https://github.com/GeoscienceAustralia/anuga_core
(5) Install anuga
(7) Completed


<font size=5> Step #3: Set up working directories<font> 


<font size=3> We will access data files stored on a shared Google Drive. You will also need to save files to your own Google Drive.<font>



In [ ]:
path = where_to_mount + 'MyDrive/processing/'
path_code = path + 'code/'
path_templates = path +  'templates/'
path_configs = path + 'configs/'
path_ancillary = path + 'ancillary/'
sys.path.insert(1,path_code)


<font size=5> Step #4: Now we will import Python packages, which will be used for future tutorials. If there are any errors, try rerunning the first cell <font> 




In [ ]:
import sys
import os
import pandas as pd
import shutil
from datetime import datetime
from string import Template
import fnmatch
import geopandas as gpd
import rasterio
from osgeo import gdal 

import matplotlib.pyplot as plt

import anuga
import ee 
import geemap
from pathlib import Path

In [ ]:
from BYOM_extra import (make_segments_for_swot)
from BYOM_Utilities_V1 import (make_watermask, build_directory)


/content/drive/MyDrive/processing/code/use_orinoco.py:39: UserWarning: ShapelyDeprecationWarning
  warnings.warn('ShapelyDeprecationWarning')
/content/drive/MyDrive/processing/code/use_orinoco.py:40: UserWarning: UserWarning
  warnings.warn('UserWarning')


In [ ]:
AOI = 'komo'
modelpath = '/content/drive/MyDrive/komo/Simulations/komo_300cms_20210101_Elevation_ocean-GEBCO_land-GLO30_above1260m-wdpower_below1260m-plane_wetland-uni0m_lakes-uni1_Meshes_base_100000m2_BrBout'
scenario = modelpath.split('/')[-1]
mesh_path = folders[3] + scenario.split('_')[-4:-2]

res = 30 #meters
print('\n')
print('Study area is ' + AOI)
print('Resolution of this setup is %sm' %(res))

working_path,folders = build_directory(mounted_drive, AOI)


Elevation_ocean-GEBCO_land-GLO30_above21383m-wdpower_below21383m-plane_wetland-uni0m_lakes-uni1m


In [ ]:
parameters = pd.read_csv(folders[6] + '/config_%s.csv' %(AOI))


In [ ]:
## STEP 5
## Make river and ocean segments for SWOT simulator
## Approximately 900m for river segments and 10000m2 for oceans
ulx  = parameters['ulx'][0]                                                 # ULX coordinate
lry  = parameters['lry'][0]                                                 # LRY coordinate
lrx  = parameters['lrx'][0]                                                 # LRX coordinate
uly  = parameters['uly'][0]                                                 # ULY coordinate
EPSG = int(parameters['EPSG'][0])
startdate      = (parameters['StartDate'][0])                    # Date to start simulation
    enddate        = '20210105'#(parameters['EndDate'][0])                      # Date to start simulation
    base_scale     = int(parameters['BaseScale'][0])
        end    = (datetime.datetime.strptime(enddate,'%Y%m%d')-datetime.datetime.strptime(startdate,'%Y%m%d')).days*24


try:
    dem = rasterio.open(elevationpath + '/' + elev_name + '_' + str(swot_res) + '.tif')
except:
    os.system('gdalwarp -overwrite -tr %s %s %s/%s_%s.tif %s/%s_%s_%s.tif '\
                  ' -te %s %s %s %s -srcnodata -9999 -dstnodata -9999 -co COMPRESS=DEFLATE -q'
                  %(swot_res,swot_res,elevationpath,AOI,elev_name,elevationpath,AOI,elev_name,swot_res,ulx,lry,lrx,uly))
    dem = rasterio.open(elevationpath + '/' + AOI + '_' + elev_name + '_' + str(swot_res) + '.tif')



In [ ]:
extentfile = gpd.read_file('%s%s_input.shp' %(folders[0],AOI))
extentfile.crs = 'EPSG:4326'

Path('%s/outputRST/abselev'%(modelpath)).mkdir(parents=True, exist_ok=True)
Path('%s/outputRST/depth'%(modelpath)).mkdir(parents=True, exist_ok=True)
Path('%s/outputRST/stage'%(modelpath)).mkdir(parents=True, exist_ok=True)
Path('%s/outputRST/waterelev'%(modelpath)).mkdir(parents=True, exist_ok=True)
Path('%s/outputRST/flooded'%(modelpath)).mkdir(parents=True, exist_ok=True)

print('Extract bed elevation from SWW file')
print('Resampled to %s m resolution' %(xres))
Make_Geotif(swwFile=modelpath +'/'+ scenario + '.sww',
          output_quantities=['elevation'],
          myTimeStep=0,
          CellSize=res,
          lower_left=None, upper_right=None,
          EPSG_CODE=EPSG,
          proj4string=None,
          velocity_extrapolation=True,
          min_allowed_height=1.0e-05,
          output_dir=modelpath + 'outputRST/',
          k_nearest_neighbours=1,
          bounding_polygon=anuga.read_polygon('%s%s_domain_points.csv' %(mesh_path,AOI)),
          verbose=True,creation_options=['COMPRESS=DEFLATE'])

print('\nExtract time series of stage from SWW file')
Make_Geotif(swwFile='%s/%s.sww' %(modelpath,scenario),
          output_quantities=['stage'],
          myTimeStep=range(1,3,1),
          CellSize=res,
          lower_left=None, upper_right=None,
          EPSG_CODE=EPSG,
          proj4string=None,
          velocity_extrapolation=True,
          min_allowed_height=1.0e-05,
          output_dir=modelpath + 'outputRST/stage/',
          k_nearest_neighbours=1,
          bounding_polygon=anuga.read_polygon('%s%s_domain_points.csv' %(mesh_path,AOI)),
          verbose=True,creation_options=['COMPRESS=DEFLATE'])

CNES SWOT simulator files will be here: /Volumes/FortressL3/ANUGA/GlobalDeltas/examples/gabon/Scenarios/Elevation_ocean-GEBCO_land-GLO30_above21383m-wdpower_below21383m-plane_wetland-uni0m_lakes-uni1m/Meshes_base_50000m2/gabon_237cms_20210101_BoutBrBrBr_20220915//SWOT_simulator
